# Zufallssuche, hill climbing und hill climbing with restarts für min makespan

Dieses Notebook vergleicht Zufallssuche, hill climbing und hill climbing mit restarts für das Problem des *minimum makespans* mit zwei Maschinen. 

Zunächst führen wir die nötigen <code>import</code> durch.

In [1]:
import random
import time
import numpy as np  # scientific computing library, see numpy.org

## Zufallsinstanzen

Um die Algorithmen zu testen, brauchen wir Instanzen des Problems. Wir erzeugen einfache Zufallsinstanzen. 

In [2]:
def rnd_instance(n):
    d=[random.randint(1001,100000) for _ in range(n)]
    d2=[dd-random.randint(-1000,1000) for dd in d]
    return np.array([d,d2])/100

d=rnd_instance(10)
d

array([[429.23, 680.96, 276.56, 811.91, 829.31, 506.5 , 182.44, 598.91,
        990.29, 330.01],
       [439.  , 680.98, 273.05, 810.89, 826.19, 498.16, 191.63, 599.09,
        996.81, 330.76]])

Erste Zeile zeigt die Dauern der Aufträge auf der ersten Maschine; die zweite, die Dauern auf der zweiten Maschine.

Was wir noch brauchen: Eine Methode, um den makespan zu berechnen. Dazu verwenden wir wieder die Möglichkeit bei der Erstellung von Listen mit <code>if</code> zu filtern. Die Methode erwartet eine Zuteilung <code>assignment</code>: Dies ist einfach eine 0,1-Liste, wobei der Eintrag 0 den jeweiligen Auftrag der ersten Maschine zuweist und 1 der zweiten Maschine.

In [3]:
def compute_makespan(assignment,instance):
    """
    expects assignment to be a list that maps jobs to machines (machine 0 or machine 1), ie assignment is to be a list with 0/1 entries.
    """
    T1=sum([d for i,d in enumerate(instance[0,:]) if assignment[i]==0])
    T2=sum([d for i,d in enumerate(instance[1,:]) if assignment[i]==1])
    return max(T1,T2)

Wir testen die Methode anhand einer Zufallszuteilung:

In [4]:
n=d.shape[1]
rnd_solution=[random.randint(0,1) for _ in range(n)]
makespan=compute_makespan(rnd_solution,d)
print("Zufallslösung:")
print(rnd_solution)
print("Makespan: {:.1f}".format(makespan))

Zufallslösung:
[0, 1, 0, 1, 1, 0, 0, 0, 1, 0]
Makespan: 3314.9


## Zufallssuche

Als erstes erzeugen wir brutal viele Zufallslösungen und wählen dann diejenige mit kleinstem makespan aus.

Die folgende Klasse ist nur Bequemlichkeit: Sie ermöglicht ein einfaches Tracken der bisher besten Lösung.

In [5]:
class Best_Tracker:
    def __init__(self):
        self.reset()
        
    def reset(self):
        """setzt tracker zurück"""
        self.best_cost=np.inf
        self.best=None
    
    def update(self,solution,cost):
        """merke Lösungskosten, wenn sie geringer sind als bisher beste Lösung"""
        if cost<self.best_cost:
            self.best_cost=cost
            self.best=solution.copy()

Wir erzeugen Zufallslösungen bis das Zeitbudget aufgebraucht ist. 

In [6]:
def best_rnd_solution(instance,time_budget):
    start_time=time.time()
    n=instance.shape[1]
    tracker=Best_Tracker()
    while time.time()-start_time<time_budget:
        # erzeuge Zufallslösung
        solution=[random.randint(0,1) for _ in range(n)]
        makespan=compute_makespan(solution,instance)
        tracker.update(solution,makespan)
    return tracker.best,tracker.best_cost

## hill climbing

Als nächstes implementieren wir hill climbing. Die lokale Veränderung ist dabei denkbar einfach: Wir wählen einen Auftrag zufällig aus und verschieben ihn auf die andere Maschine. 

Wir brechen das hill climbing ebenfalls nach vorgegebener Zeit ab, um Vergleichbarkeit mit den anderen Algorithmen zu schaffen.

In [7]:
def tweak(solution):
    """
    performs simple local change: randomly one job is transferred from one machine to the other.
    """
    position=random.randrange(len(solution))
    solution[position]=1-solution[position]
    return position

def undo_tweak(solution,position):
    """
    swaps the job at position back 
    """
    solution[position]=1-solution[position]    

def hill_climbing(solution,instance,time_budget,tries=20):
    start_time=time.time()
    step_tracker=Best_Tracker()   # keeps track of best solution during single step
    current_cost=compute_makespan(solution,instance)
    while time.time()-start_time<time_budget:
        step_tracker.reset()
        for _ in range(tries):         # try several local changes and pick the best one
            position=tweak(solution)
            ms=compute_makespan(solution,instance)  # this could be done more efficiently -- we do not actually have to recompute the makespan in each step
            step_tracker.update(solution,ms)
            undo_tweak(solution,position) 
        if step_tracker.best_cost<current_cost:    # we found an improvement, so we take the step 
            solution=step_tracker.best             # if not, we simply try again to find an improvement until time runs out
            current_cost=step_tracker.best_cost
    return solution,current_cost

## hill climbing mit random restarts

Schließlich fügen wir noch restarts hinzu. Einen Teil des hill climbing-Codes können wir wiederverwenden. Das Zeitbudget des hill climbings stellen wir so ein, dass 10 restarts gemacht werden. Dies sollte eigentlich ein Einstellparameter sein.

In [8]:
def hill_climbing_random_restarts(instance,time_budget,tries=20):
    start_time=time.time()
    n=instance.shape[1]
    tracker=Best_Tracker()
    while time.time()-start_time<time_budget:
        solution=[random.randint(0,1) for _ in range(n)]
        makespan=compute_makespan(solution,instance)
        hill_climbing_time_budget=min(time_budget/10,time.time()-start_time)  # div by 10 is arbitrary
        solution,makespan=hill_climbing(solution,instance,hill_climbing_time_budget)
        tracker.update(solution,makespan)
    return tracker.best,tracker.best_cost   

## Vergleich

Schließlich vergleichen wir die drei Algorithmen. Achtung, das Ausführen dieser Zelle dauert ein wenig.

In [9]:
repeats=50
n=100
time_budget=3
results=[0]*3
for _ in range(repeats):
    instance=rnd_instance(n)
    _,rnd=best_rnd_solution(instance,time_budget)
    _,hill=hill_climbing([0]*n,instance,time_budget)
    _,rnd_hill=hill_climbing_random_restarts(instance,time_budget)
    results[np.argmin([rnd,hill,rnd_hill])]+=1   # we count who wins
    
print("Niedrigster makespan in {} Versuchen:".format(repeats))
print("Zufallssuche                  : {:2.1f}%".format(results[0]/repeats*100))
print("hill climbing                 : {:2.1f}%".format(results[1]/repeats*100))
print("hill climbing, random restarts: {:2.1f}%".format(results[2]/repeats*100))

Niedrigster makespan in 50 Versuchen:
Zufallssuche                  : 84.0%
hill climbing                 : 2.0%
hill climbing, random restarts: 14.0%


Dass hier die Zufallssuche mit Abstand am erfolgreichsten ist, bedeutet nicht, dass Zufallssuche generell das Mittel der Wahl ist. Vielmehr ist min makespan (mit zwei Maschinen) ein so einfaches Problem, dass es sich offenbar lohnt den Lösungsraum weit zu erkunden. Zudem hat hill climbing noch Parameter, die vielleicht besser eingestellt werden könnten (wie viele lokale Änderungen sollen ausprobiert werden, bevor der lokale Schritt getan wird?). Das gleiche gilt für hill climbing mit random restarts.